In this notebook we implement the proof-of-concept of allocator based on (address, size) + (size, address) model.

In [3]:
import bisect

In [55]:
class Allocator:
    def __init__(self, size = 1000):
        self.size = size
        # allocated addresses, sorted tuples (address, size)
        self.allocs = []
        # blank-index, sorted tuples (size, address)
        # initialize with all space available initially
        self.blanks = [(self.size, 0)]        
    
    def alloc(self, size) -> int:
        # find the 1st blank of sufficient size
        index = bisect.bisect_left(self.blanks, (size, 0))
        if index == len(self.blanks):
            raise Exception("Insufficient memory")
        
        blank = self.blanks.pop(index)
        # take first 'size' bytes from blank
        new_alloc = (blank[1], size)
        remaining = (blank[0] - size, blank[1] + size)
        # use insort to keep the lists sorted
        bisect.insort(self.allocs, new_alloc)
        if remaining[0] > 0:
            bisect.insort(self.blanks, remaining)
        return new_alloc[0]
        
    def dealloc(self, address):
        # find the allocation first
        index = bisect.bisect_left(self.allocs, (address, 0))
        if index == len(self.allocs) or self.allocs[index][0] != address:
            raise Exception(f"Invalid address: {address}")
        
        alloc = self.allocs[index]
        
        # collect the neighboring blanks
        b0, b1 = None, None
        if index > 0:
            # compare with the left-neighbor
            b0_size = self.allocs[index][0] - self.allocs[index - 1][0] - self.allocs[index - 1][1]
            if b0_size > 0:
                b0 = self.allocs[index - 1][0] + b0_size, b0_size
        elif alloc[0] > 0:
            # 1st allocation but there's blank space left of it
            b0 = (0, alloc[0])
        
        if index < len(self.allocs) - 1:
            # compare with the right-neighbor
            b1_size = self.allocs[index + 1][0] - self.allocs[index][0] - self.allocs[index][1]
            if b1_size > 0:
                b1 = self.allocs[index + 1][0] - b1_size, b1_size
        elif alloc[0] + alloc[1] < self.size:
            # last allocation but there's blank space right of it
            b1 = (alloc[0] + alloc[1], self.size - alloc[0] - alloc[1])
        
        # remove blanks
        if b0 is not None:
            del self.blanks[bisect.bisect_left(self.blanks, b0)]
        if b1 is not None:  
            del self.blanks[bisect.bisect_left(self.blanks, b1)]
        
        # remove the allocation
        del self.allocs[index]
        
        # re-insert a merged blank
        b0 = alloc if b0 is None else b0
        b1 = alloc if b1 is None else b1
        blank = (b0[0], b1[0] + b1[1] - b0[0])
        bisect.insort(self.blanks, blank)
    
    def get_allocation_size(self, address):
        index = bisect.bisect_left(self.allocs, (address, 0))
        if index == len(self.allocs) or self.allocs[index][0] != address:
            raise Exception(f"Invalid address: {address}")
        return self.allocs[index][1]
    
    def __repr__(self):
        return f"blanks:{'+'.join([str(b) for b in self.blanks])}"

In [56]:
def allocator_tests():
    a = Allocator()
    my_objects = []
    # 3 allocations of size '3'
    for _ in range(3):
        my_objects.append(a.alloc(3))
    
    print(a)
    # dealocate in this order: 2nd, 3rd, 1st
    a.dealloc(my_objects.pop(1))
    print(a)
    a.dealloc(my_objects.pop(1))
    print(a)
    a.dealloc(my_objects.pop(0))
    print(a)

In [57]:
allocator_tests()

blanks:(991, 9)
blanks:(3, 3)+(991, 9)
blanks:(3, 997)
blanks:(0, 1000)
